In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import os
import numpy as np
from src import config
from src.preprocessing import _load_features, _load_labels, _remove_duplicate_labels

In [4]:
%cd /home/marco/speaker_identification
features = _load_features(first_n_files=4)

/home/marco/speaker_identification


In [18]:
df_labels = _load_labels()
df_labels = _remove_duplicate_labels(df_labels)

In [19]:
labels

,filename,label_date,label_time,label
0,PTT-20200113-WA0016,20220824,101700,Person 1
1,PTT-20200116-WA0000,20220824,101730,Person 2
2,PTT-20200117-WA0007,20220824,101800,Person 2
3,PTT-20200119-WA0013,20220824,101830,Person 1
